In [1]:
import numpy as np
import pandas as pd
import datetime
import re

import gensim.downloader as api
word_vectors = api.load("glove-wiki-gigaword-200")

import itertools
from tqdm.auto import tqdm

pd.options.display.max_rows = 999

In [2]:
DATA_PATH = '../Data/'
df_max = pd.read_csv(DATA_PATH + 'clean_max.csv', index_col=0, parse_dates=[3])
df_mar = pd.read_csv(DATA_PATH + 'clean_mar.csv', index_col=0, parse_dates=[3])
df_sam = pd.read_csv(DATA_PATH + 'clean_sam.csv', index_col=0, parse_dates=[3])

df_all = [df_max, df_mar, df_sam]
df_sizes = [df.shape[0] for df in df_all]

### Assign Data Types 
int_columns = ['ID', 'Session', 'Receipt', 'Quantity', 'Basket']
string_columns = ['Item', 'ItemMore', 'Category', 'Comment']
for df in df_all:
    df.loc[:, int_columns] = df[int_columns].astype('int16')
    df.loc[:, string_columns] = df[string_columns].astype(str)
    df.loc[:, 'Date'] = pd.to_datetime(df.Date, errors='coerce').dt.date
    
### Shared Participants
ids_shared = set(df_max.ID.unique()) & set(df_mar.ID.unique()) & set(df_sam.ID.unique())

df_shared = []
for df in df_all:
    df_shared.append(df[df.ID.isin(ids_shared)].copy())

In [3]:
def compare(df):
    result = []
    df_split = df.str.split()
    
    # return shared words
    shared_words = list(set.intersection(*map(set, df_split)))
    if shared_words:
        for word in shared_words:
            result.append(word)
        df_unique = [[word for word in phrase if word not in shared_words] for phrase in df_split]
    else:
        df_unique = df_split
    
    # discard word outside vocabulary
    df_in_voc = [[word for word in phrase if word in word_vectors.vocab] for phrase in df_unique]
    
    # if any text is exhausted return
    if not all(df_in_voc):
        df['WordVec'] = ' '.join(result)
        df['Similarity'] = 1.0
        return df
    
    # use word vectors to average remaining words
    words = [word for text in df_in_voc for word in text]
    wv_result = word_vectors.most_similar(positive=[*words])
    most_similar_key, similarity = wv_result[0]
    result.append(most_similar_key)
    df['WordVec'] = ' '.join(result)
    df['Similarity'] = round(similarity, 3)
    return df

In [4]:
def align(df1, df2):
    # equalize lengths
    df1_length, df2_length = len(df1), len(df2)
    if df1_length > df2_length:
        length_diff = df1_length - df2_length
        df2 = df2.reindex(df2.index.union(list(range(df2_length, df2_length + length_diff))))
    elif df2_length > df1_length:
        length_diff = df2_length - df1_length
        df1 = df1.reindex(df1.index.union(list(range(df1_length, df1_length + length_diff))))
    df1 = df1.fillna('')
    df2 = df2.fillna('')
    assert len(df1) == len(df2)
    
    # remove identical matches
    result_pairs = []
    df1_dropped = df1.copy()
    df2_dropped = df2.copy()
    for df1_idx, df1_word in enumerate(df1):
        matches = df2_dropped.str.contains(df1_word, regex=False)
        if any(matches):
            first_match_index = matches.idxmax()
            result_pairs.append((df1_idx, first_match_index))
            df1_dropped.drop(index=df1_idx, inplace=True)
            df2_dropped.drop(index=first_match_index, inplace=True)

    # all permutations of remaining indices
    perms = list(itertools.permutations(df2_dropped.index))
    print(len(perms))
    
    # generate word vectors and similarity
    total_similarity = []
    if len(perms) > 1:
        for p in tqdm(perms, desc="Permutations", leave=False):
            df = pd.concat([df1_dropped.reset_index(drop=True), df2_dropped.reindex(p).reset_index(drop=True)], axis=1)
            df = df.apply(compare, axis=1)
            # sum similarity 
            total_similarity.append(sum(df.Similarity))
    
        # find max permutation
        result_index = perms[total_similarity.index(max(total_similarity))]
    else:
        result_index = perms[0]
    
    # return concatendated dataframe with word vectors
    top_index_left, top_index_right = zip(*result_pairs)
    df_top = pd.concat([df1.reindex(top_index_left).reset_index(drop=True), 
                        df2.reindex(top_index_right).reset_index(drop=True)], axis=1, ignore_index=True)
    
    bot_index_left = df1_dropped.index
    bot_index_right = result_index
    df_bot = pd.concat([df1.reindex(bot_index_left).reset_index(drop=True), 
                        df2.reindex(bot_index_right).reset_index(drop=True)], axis=1, ignore_index=True)
    
    df_combined = pd.concat([df_top, df_bot], ignore_index=True)
    return df_combined

In [5]:
### Align Data Sets by hand
# ID 137 and 114 have low variation
df1 = df_shared[0].loc[df_shared[0].ID == 137, 'Item'].copy().reset_index(drop=True)
df2 = df_shared[1].loc[df_shared[1].ID == 137, 'Item'].copy().reset_index(drop=True)
df3 = df_shared[2].loc[df_shared[2].ID == 137, 'Item'].copy().reset_index(drop=True)

# align by inspecting for proof of concept
df3_aligned = df3.drop([102]).reset_index(drop=True)
df_hand_aligned = pd.concat([df1, df3_aligned], axis=1)
df_hand_aligned.columns = ['Item1', 'Item3']

df_hand_aligned = df_hand_aligned.apply(compare, axis=1)
display(df_hand_aligned)

,Item1,Item3,WordVec,Similarity
0,nan,nan,nan,1.000
1,cleaning wipes,glass wipes,wipes kitchen,0.621
2,cleaning spray,spray bottle,spray bottles,0.666
3,nan,nan,nan,1.000
4,book,children's book,book,1.000
5,slippers,slippers,slippers,1.000
6,tissues,tissues,tissues,1.000
7,tape,tape,tape,1.000
8,video creator,video maker kit,video manufacturer,0.660
9,dill pickles,dill pickles,dill pickles,1.000


In [6]:
### Test hand alignment against algorithm

#12 baskets
df_final = pd.DataFrame()
for basket in tqdm(range(1,2), desc="Basket"):
    df1 = df_shared[0].loc[(df_shared[0].ID == 137) & (df_shared[0].Basket == basket), 'Item'].copy().reset_index(drop=True)
    df3 = df_shared[2].loc[(df_shared[2].ID == 137) & (df_shared[2].Basket == basket), 'Item'].copy().reset_index(drop=True)
    df_aligned = align(df1, df3)
    display(df_aligned)
    #df_final = pd.concat([df_final, df_aligned], ignore_index=True)

#display(df_final)

Basket:   0%|          | 0/1 [00:00<?, ?it/s]

120


Permutations:   0%|          | 0/120 [00:00<?, ?it/s]

,0,1
0,nan,nan
1,nan,nan
2,book,children's book
3,slippers,slippers
4,tissues,tissues
5,tape,tape
6,dill pickles,dill pickles
7,penne,penne pasta
8,carrots,carrots
9,peanuts,peanuts


In [7]:
display(pd.concat([df_shared[0].loc[(df_shared[0].ID == 137) & (df_shared[0].Basket == 7), 'Item'].reset_index(drop=True),
                   df_shared[2].loc[(df_shared[2].ID == 137) & (df_shared[2].Basket == 7), 'Item'].reset_index(drop=True)],
                   axis=1, ignore_index=True))

,0,1
0,potato chips,potato chips
1,italian salad dressing,italian salad dressing
2,salad dressing,salad dressing
3,dill pickles,dill pickles
4,dill kosher pickles,dill kosher pickles
5,soda (coca cola0,"cola, zero soda"
6,cranberry mango juice,cranberry mango juice
7,pinto beans,pinto beans
8,kidney beans,kidney beans
9,red beans,red beans
